<a href="https://colab.research.google.com/github/AldataSci/CapstoneData/blob/main/Cluster_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is gonna be the final part of my project in which I will attempt clustering, I am gonna split the datasets into 2022 post pandemic and 2019 pre pandemic. I am going to attempt to do some regex-cleaning so I can merge the 2022 ridership and 2019 ridership with the original df.

Edit: It took a while and a lot of cleaning and searching but FUZZYWUZZY package helped me matched the station names

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
## This is the dataframe that contains the complaints...
df = pd.read_csv('https://raw.githubusercontent.com/AldataSci/CapstoneData/refs/heads/main/cleaned_stations_df.csv')

# Let's remove station_name all NULL..

clean_df_2022 = df[df['year'] == 2022]

# Create A pre-pandemic chart
clean_df_2019 = df[df['year'] == 2019]

In [ ]:
clean_df_2019.shape

(12001, 26)

In [ ]:
clean_df_2022.shape

(11242, 26)

In [ ]:
## Hmm what do I do the value_counts right.. by station?

grouped_2019 = clean_df_2019.groupby('station_name').size().reset_index(name = 'complaint_count')
grouped_2022 = clean_df_2022.groupby('station_name').size().reset_index(name = 'complaint_count')


In [ ]:
# Merge the df

merged_df = pd.merge(clean_df_2022, grouped_2022, on = 'station_name', how = 'left')
merged_df_2019 = pd.merge(clean_df_2019, grouped_2019, on = 'station_name', how = 'left')
## I will assume everything is correct..

In [ ]:
ridership_station_df = pd.read_csv('https://raw.githubusercontent.com/AldataSci/CapstoneData/refs/heads/main/2023%20Subway%20Tables.csv',header = 1)

ridership_station_df.head(5)

,Station (alphabetical by borough),*,Boro,2018,2019,2020,2021,2022,2023,2022-2023 Change,Unnamed: 10,2023 Rank
0,The Bronx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"138 St-Grand Concourse (4,5)",1.0,Bx,944598.0,1035878.0,371408.014,656866.0,766610.0,#NAME?,18661,2.4342233991208%,#NAME?
2,"149 St-Grand Concourse (2,4,5)",NaN,Bx,3972763.0,3931908.0,1815784.971,1832521.0,2026363.0,#NAME?,61416,3.03084886567708%,#NAME?
3,"161 St-Yankee Stadium (B,D,4)",NaN,Bx,8392290.0,8254928.0,3221650.993,4077604.0,5023193.0,#NAME?,#NAME?,#NAME?,#NAME?
4,167 St (4),NaN,Bx,2933140.0,2653237.0,1396286.968,1615072.0,1847368.0,#NAME?,#NAME?,#NAME?,#NAME?


Data-Cleaning For Ridership

In [ ]:
## Let's remove the empty columns for now I will just get the 2023 dataset?

# Produce a copy of the dataframe/

riders_station_copy = ridership_station_df.copy()

df_2022 = riders_station_copy[['Station (alphabetical by borough)','2022']]

df_2019 = riders_station_copy[['Station (alphabetical by borough)','2019']]

In [ ]:
# For now let's remove the NA observation..

df_2019.dropna(inplace = True)
df_2022.dropna(inplace = True)

<ipython-input-10-9c4f857db677>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2019.dropna(inplace = True)
<ipython-input-10-9c4f857db677>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2022.dropna(inplace = True)


In [ ]:
# The last observation is systems_totals should I get rid of it?

# Meh I also get rid of the borough counts as well hopefully it is just station names and their ridership
df_station_2022 = df_2022.iloc[:-5]
df_station_2019 = df_2019.iloc[:-5]

In [ ]:
# Let's take a look at the unique station_name

df_station_2022['Station (alphabetical by borough)'].unique()

array(['138 St-Grand Concourse (4,5)', '149 St-Grand Concourse (2,4,5)',
       '161 St-Yankee Stadium (B,D,4)', '167 St (4)', '167 St (B,D)',
       '170 St (4)', '170 St (B,D)', '174 St (2,5)', '174-175 Sts (B,D)',
       '176 St (4)', '182-183 Sts (B,D)', '183 St (4)', '219 St (2,5)',
       '225 St (2,5)', '231 St (1)', '233 St (2,5)', '238 St (1)',
       '3 Av-138 St (6)', '3 Av-149 St (2,5)', 'Allerton Av (2,5)',
       'Baychester Av (5)', 'Bedford Park Blvd (B,D)',
       'Bedford Park Blvd-Lehman College (4)', 'Bronx Park East (2,5)',
       'Brook Av (6)', 'Buhre Av (6)', 'Burke Av (2,5)',
       'Burnside Av (4)', 'Castle Hill Av (6)', 'Cypress Av (6)',
       "East 143 St-St Mary's St (6)", 'East 149 St (6)',
       'East 180 St (2,5)', 'Eastchester-Dyre Av (5)', 'Elder Av (6)',
       'Fordham Rd (4)', 'Fordham Rd (B,D)', 'Freeman St (2,5)',
       'Gun Hill Rd (2,5)', 'Gun Hill Rd (5)', 'Hunts Point Av (6)',
       'Intervale Av (2,5)', 'Jackson Av (2,5)', 'Kingsbridge R

# Data-Cleaning
Data-Cleaning and Then Fix Up the Station Name and then Fix It..

I have 2 datasets with station name but they are in different format



In [ ]:
merged_df['station_name'].unique()

array(['BROADWAY/NASSAU', '174-175 STREETS', '149 ST.-GRAND CONCOURSE',
       'PROSPECT AVENUE', '96 STREET', 'STILLWELL AVENUE-CONEY ISLAND',
       'GRAND AVE.-NEWTON', 'GUN HILL ROAD', 'BROOKLYN BRIDGE-CITY HALL',
       'FORDHAM ROAD', 'LAWRENCE STREET', '125 STREET',
       'QUEENSBORO PLAZA', '50 STREET', 'WINTHROP STREET', '42 STREET',
       '96TH STREET', 'BRIGHTON BEACH', 'UNION SQUARE',
       '3 AVENUE-149 STREET', 'BROOK AVENUE', 'ROCKAWAY AVENUE',
       '242 ST.-VAN CORTLANDT PARK', '145 STREET', '71 AVE.-FOREST HILLS',
       'KINGSBRIDGE ROAD', '59 STREET', 'WEST 34 STREET/HUDSON YARDS',
       '168 ST.-WASHINGTON HTS.', 'LEXINGTON AVE.', '116 STREET',
       '25 STREET', 'BOWERY', 'MOSHOLU PKWY.', 'UTICA AVENUE',
       'EUCLID AVENUE', '80 STREET', '14 STREET',
       '42 ST.-PORT AUTHORITY BUS TERM', '1 AVENUE',
       'UTICA AVE.-CROWN HEIGHTS', 'ASTORIA BLVD.', 'WOODLAWN',
       'W. 4 STREET', 'LONGWOOD AVENUE', 'HALSEY STREET',
       '42 ST.-TIMES SQUARE', '46

In [ ]:
## Let's clean the complaint dataset first

import re

def clean_complaints_station(station: str) -> str:
    """Clean station names from the complaints dataset."""
    # Replace slashes with spaces or hyphens
    station = station.replace('-', ' ')  # You can change this to '-' if you prefer
    # Remove extra spaces
    station = re.sub(r'\s+', ' ', station)
    # Convert to uppercase
    station = station.upper().strip()
    # Standardize abbreviations
    station = re.sub(r'\b(STREET|STREETS|STS)\b', 'ST', station)  # Convert 'ST.' to 'STREET'

    ## Convert Avenue
    station = re.sub(r'\b(AVENUE|AVENUES)\b', 'AV', station)

    # Remove TH from stations

    station = re.sub(r'\b(TH)\b', '', station)

    return station

merged_df['station_name'] = merged_df['station_name'].apply(clean_complaints_station)
merged_df_2019['station_name'] = merged_df_2019['station_name'].apply(clean_complaints_station)

merged_df['station_name'].unique()

array(['BROADWAY/NASSAU', '174 175 ST', '149 ST. GRAND CONCOURSE',
       'PROSPECT AV', '96 ST', 'STILLWELL AV CONEY ISLAND',
       'GRAND AVE. NEWTON', 'GUN HILL ROAD', 'BROOKLYN BRIDGE CITY HALL',
       'FORDHAM ROAD', 'LAWRENCE ST', '125 ST', 'QUEENSBORO PLAZA',
       '50 ST', 'WINTHROP ST', '42 ST', '96TH ST', 'BRIGHTON BEACH',
       'UNION SQUARE', '3 AV 149 ST', 'BROOK AV', 'ROCKAWAY AV',
       '242 ST. VAN CORTLANDT PARK', '145 ST', '71 AVE. FOREST HILLS',
       'KINGSBRIDGE ROAD', '59 ST', 'WEST 34 ST/HUDSON YARDS',
       '168 ST. WASHINGTON HTS.', 'LEXINGTON AVE.', '116 ST', '25 ST',
       'BOWERY', 'MOSHOLU PKWY.', 'UTICA AV', 'EUCLID AV', '80 ST',
       '14 ST', '42 ST. PORT AUTHORITY BUS TERM', '1 AV',
       'UTICA AVE. CROWN HEIGHTS', 'ASTORIA BLVD.', 'WOODLAWN', 'W. 4 ST',
       'LONGWOOD AV', 'HALSEY ST', '42 ST. TIMES SQUARE', '46 ST',
       '7TH AV', 'DEKALB AV', 'PROSPECT PARK', 'EAST 177 ST. PARKCHESTER',
       '23 ST', 'ATLANTIC AV', 'GATES AV', 'BUHRE

In [ ]:
## Now let's edit this out now NOTE I HAVE TO tweak certain stuff because the naming convention is weird..

def clean_ridership_station(station: str) -> str:
    """Clean station names from the ridership dataset."""
    # Remove the train lines (anything inside parentheses)
    station = re.sub(r'\(.*\)', '', station)
    # Replace hyphens with spaces
    station = station.replace('-', ' ')  # You can change this to '-' if you prefer
    # Remove extra spaces
    station = re.sub(r'\s+', ' ', station)
    # Strip leading/trailing spaces and convert to uppercase
    station = station.strip().upper()

    ## Remove Special Characters

    station = re.sub(r'[^a-zA-Z0-9\s]', '', station)

    ## Convert STREETS
    station = re.sub(r'\b(STREET|STREETS|STS)\b', 'ST', station)

    ## Convert CENTER to CTR

    station = re.sub(r'\b(CENTER)\b', 'CTR', station)

    ## Fix West 4 STREET
    station = re.sub(r'\b(WEST 4 ST WASHINGTON SQ)\b', 'W. 4 ST', station)

    return station

df_station_2022['Station (alphabetical by borough)'] = df_station_2022['Station (alphabetical by borough)'].apply(clean_ridership_station)
df_station_2019['Station (alphabetical by borough)'] = df_station_2019['Station (alphabetical by borough)'].apply(clean_ridership_station)

df_station_2022['Station (alphabetical by borough)'].unique()

<ipython-input-15-199a8840c49a>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_station_2022['Station (alphabetical by borough)'] = df_station_2022['Station (alphabetical by borough)'].apply(clean_ridership_station)
<ipython-input-15-199a8840c49a>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_station_2019['Station (alphabetical by borough)'] = df_station_2019['Station (alphabetical by borough)'].apply(clean_ridership_station)


array(['138 ST GRAND CONCOURSE', '149 ST GRAND CONCOURSE',
       '161 ST YANKEE STADIUM', '167 ST', '170 ST', '174 ST',
       '174 175 ST', '176 ST', '182 183 ST', '183 ST', '219 ST', '225 ST',
       '231 ST', '233 ST', '238 ST', '3 AV 138 ST', '3 AV 149 ST',
       'ALLERTON AV', 'BAYCHESTER AV', 'BEDFORD PARK BLVD',
       'BEDFORD PARK BLVD LEHMAN COLLEGE', 'BRONX PARK EAST', 'BROOK AV',
       'BUHRE AV', 'BURKE AV', 'BURNSIDE AV', 'CASTLE HILL AV',
       'CYPRESS AV', 'EAST 143 ST ST MARYS ST', 'EAST 149 ST',
       'EAST 180 ST', 'EASTCHESTER DYRE AV', 'ELDER AV', 'FORDHAM RD',
       'FREEMAN ST', 'GUN HILL RD', 'HUNTS POINT AV', 'INTERVALE AV',
       'JACKSON AV', 'KINGSBRIDGE RD', 'LONGWOOD AV', 'MIDDLETOWN RD',
       'MORRIS PARK', 'MORRISON AV SOUNDVIEW', 'MOSHOLU PKWY',
       'MT EDEN AV', 'NEREID AV', 'NORWOOD 205 ST', 'PARKCHESTER',
       'PELHAM BAY PARK', 'PELHAM PKWY', 'PROSPECT AV', 'SIMPSON ST',
       'ST LAWRENCE AVE', 'TREMONT AV', 'VAN CORTLANDT PARK 242 

In [ ]:
df_station_2019

,Station (alphabetical by borough),2019
1,138 ST GRAND CONCOURSE,1035878.0
2,149 ST GRAND CONCOURSE,3931908.0
3,161 ST YANKEE STADIUM,8254928.0
4,167 ST,2653237.0
5,167 ST,2734530.0
...,...,...
422,SUTPHIN BLVD ARCHER AV JFK AIRPORT,7354064.0
423,VERNON BLVD JACKSON AV,4623070.0
424,WOODHAVEN BLVD,1337787.0
425,WOODHAVEN BLVD,6381132.0


In [ ]:
df_combined_ridership = df_station_2022.groupby('Station (alphabetical by borough)',as_index = False).sum()
df_combined_ridership_2019 = df_station_2019.groupby('Station (alphabetical by borough)',as_index = False).sum()

# Merge the two dataframes now..



In [ ]:
final_df = pd.merge(merged_df, df_combined_ridership, left_on='station_name',right_on = 'Station (alphabetical by borough)', how='left')

final_df_2019 = pd.merge(merged_df_2019, df_combined_ridership_2019, left_on='station_name',right_on = 'Station (alphabetical by borough)', how='left')
## View the selected column and make sure I track everything
final_df[['station_name','Station (alphabetical by borough)','2022']].head(5)

,station_name,Station (alphabetical by borough),2022
0,BROADWAY/NASSAU,NaN,NaN
1,174 175 ST,174 175 ST,859988.0
2,149 ST. GRAND CONCOURSE,NaN,NaN
3,PROSPECT AV,PROSPECT AV,2408486.0
4,96 ST,96 ST,15818876.0


#FUZZYWUZZY
We use the fuzzywuzzy to match smiliar names with the station to ensure we have everything as accurate as possible

In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.9 MB/s eta 0:00:00


In [ ]:
## Let's try fuzzywuzzing matching

from fuzzywuzzy import fuzz
from fuzzywuzzy import process


def match_station(row, ridership_stations,threshold=80):
    # Use fuzzywuzzy to find the best match from the ridership DataFrame
    best_match = process.extractOne(row['station_name'], ridership_stations)
    return best_match[0] if best_match else None

# Apply the matching function to the merged_df (complaints DataFrame)
merged_df['Matched_Station'] = merged_df.apply(match_station, ridership_stations=df_combined_ridership['Station (alphabetical by borough)'], axis=1)

## Apply it to the 2019 dataset as well

merged_df_2019['Matched_Station'] = merged_df_2019.apply(match_station, ridership_stations=df_combined_ridership_2019['Station (alphabetical by borough)'], axis=1)

In [ ]:
merged_df_2019.columns

Index(['Unnamed: 0', 'cmplnt_fr_dt', 'cmplnt_fr_tm', 'cmplnt_to_tm', 'rpt_dt',
       'ofns_desc', 'crm_atpt_cptd_cd', 'law_cat_cd', 'boro_nm',
       'loc_of_occur_desc', 'prem_typ_desc', 'juris_desc', 'jurisdiction_code',
       'x_coord_cd', 'y_coord_cd', 'susp_age_group', 'susp_race', 'susp_sex',
       'transit_district', 'latitude', 'longitude', 'lat_lon', 'station_name',
       'day', 'month', 'year', 'complaint_count', 'Matched_Station'],
      dtype='object')

In [ ]:
final_df2 = pd.merge(merged_df, df_combined_ridership, left_on='Matched_Station',right_on = 'Station (alphabetical by borough)', how='left')

final_df2_2019 = pd.merge(merged_df_2019, df_combined_ridership_2019, left_on='Matched_Station',right_on = 'Station (alphabetical by borough)', how='left')

In [ ]:
final_df2_2019[['station_name','Matched_Station','Station (alphabetical by borough)','2019']]

,station_name,Matched_Station,Station (alphabetical by borough),2019
0,LEFFERTS BLVD.,OZONE PARK LEFFERTS BLVD,OZONE PARK LEFFERTS BLVD,2216647.0
1,59 ST. COLUMBUS CIRCLE,59 ST COLUMBUS CIRCLE,59 ST COLUMBUS CIRCLE,23040650.0
2,42 ST. TIMES SQUARE,103 ST,103 ST,9303988.0
3,BOWERY,BOWERY,BOWERY,1325244.0
4,BEACH 25 ST,BEACH 25 ST,BEACH 25 ST,517164.0
...,...,...,...,...
11996,JACKSON AV,JACKSON AV,JACKSON AV,1584727.0
11997,59 ST. COLUMBUS CIRCLE,59 ST COLUMBUS CIRCLE,59 ST COLUMBUS CIRCLE,23040650.0
11998,GATES AV,GATES AV,GATES AV,2249791.0
11999,161 ST. YANKEE STADIUM,161 ST YANKEE STADIUM,161 ST YANKEE STADIUM,8254928.0


Some of the stations didnt match exactly as I expected So I may delete some of the observations.

In [ ]:
stations_missing = ['51 ST','CORTLANDT ST','42 ST. TIMES SQUARE','7TH AV','AV "J"','UNION SQUARE','AV "H"','WILLETS POINT SHEA STADIUM','96TH ST','42 ST. PORT AUTHORITY BUS TERM','AV "H"','JAY ST BOROUGH HALL','EAST 177 ST. PARKCHESTER','JAY ST BOROUGH HALL','PACIFIC ST','AV "J"','AV "M"','AV "U"','86TH ST','96TH ST','PARK PLACE','AV "N"','EAST 174 ST','AV "X"','SOUNDVIEW AV','9TH ST','DISTRICT 20 OFFICE','BOROUGH HALL','9TH STREET','BEACH 67 ST','62 ST','ESSEX ST','BLEECKER ST']

Okay I have to fix West 4 Street, 42 ST Port Authority Bus Team, 42 Times Square, 7TH AV, East 177ST Parkchester, 42 Times Squares, AV N, JAY ST Borough Street, UNion SQUARE, 62 ST,Pacific ST,ESSEX ST, AV U, AV J, AV H, AV X, 96TH STREET, BLEECKER ST, 8TH ST NYU, Botanic Garden, PARK PLACE, CORTLANDT ST, DISTRICT 4 OFFICE, AV X, SOUNDVIEW AV, 9TH ST,DISTRICT 20 OFFICE,BOROUGH HALL, 9TH STREET, BEACH 67 ST, EAST 174 ST, 62 ST,

In [ ]:
clean_cluster = final_df2[~final_df2['station_name'].isin(stations_missing)]

clean_cluster_2019 = final_df2_2019[~final_df2_2019['station_name'].isin(stations_missing)]

In [ ]:
## Let's perform a quick check, seems good..

clean_cluster[clean_cluster['station_name'].isin(stations_missing)]


,Unnamed: 0,cmplnt_fr_dt,cmplnt_fr_tm,cmplnt_to_tm,rpt_dt,ofns_desc,crm_atpt_cptd_cd,law_cat_cd,boro_nm,loc_of_occur_desc,...,longitude,lat_lon,station_name,day,month,year,complaint_count,Matched_Station,Station (alphabetical by borough),2022


In [ ]:
clean_cluster_2019[clean_cluster_2019['station_name'].isin(stations_missing)]

,Unnamed: 0,cmplnt_fr_dt,cmplnt_fr_tm,cmplnt_to_tm,rpt_dt,ofns_desc,crm_atpt_cptd_cd,law_cat_cd,boro_nm,loc_of_occur_desc,...,longitude,lat_lon,station_name,day,month,year,complaint_count,Matched_Station,Station (alphabetical by borough),2019


In [ ]:
## Write it to csv just in case

clean_cluster.to_csv('clean_cluster.csv')

clean_cluster_2019.to_csv('clean_cluster_2019.csv')

## Now We save everything to CSV file and work on it in a another Colab Doc